# Week 5 webinar (part II)

+ Training a sentiment classifier with PyTorch

In [2]:
# PyTorch
import torch
# model building
import torch.nn as nn
import torch.optim as optim
# to prepare data
from torchtext import data
# sample data
from torchtext import datasets
# to iterate over the training/validation reviews
from torchtext.data import BucketIterator as bi
# random seed
from numpy.random import seed
# utilities
import time

ImportError: cannot import name 'BucketIterator' from 'torchtext.data' (/home/simone/py3Venvs/nlp/lib/python3.9/site-packages/torchtext/data/__init__.py)

# Setup

## Reproducibility of the analysis

In [12]:
my_seed = 111
torch.manual_seed(my_seed)
torch.backends.cudnn.deterministic = True

# Corpus of product (movie) reviews

## NLP pipeline

In [13]:
tokens = data.Field(tokenize='spacy', tokenizer_language='en_core_web_lg')

## Handling data labels

In [14]:
labels = data.LabelField(dtype=torch.float)

## Load data (part of torchtext)

In [15]:
train, test = datasets.IMDB.splits(tokens, labels)

### Inspecting data

In [16]:
# volume of data 
print("""
=======================================
Movie reviews in training set : {}
Movie reviews in test set     : {}
---------------------------------------
""".format(len(train), len(test)))


Movie reviews in training set : 25000
Movie reviews in test set     : 25000
---------------------------------------



In [17]:
# example of movie review
print(vars(train.examples[10]))

{'text': ['Anyone', 'who', 'does', 'not', 'find', 'this', 'movie', 'funny', ',', 'does', 'not', 'understand', 'simple', 'comedy', '.', 'This', 'movie', 'is', 'not', 'a', 'complex', 'comedy', ',', 'it', 'is', 'full', 'of', 'one', 'liners', ',', 'and', 'sight', 'gags', ',', 'and', 'will', 'make', 'anyone', 'who', 'wants', 'to', 'laugh', ',', 'laugh', '...', 'The', 'alien', 'who', 'is', 'doing', 'a', 'Nicholson', 'impression', 'will', 'crack', 'you', 'up', '!'], 'label': 'pos'}


## Building a validation set

In [18]:
train, valid = train.split(random_state = seed(my_seed))

Again, we'll view how many examples are in each split.

In [19]:
# volume of data 
print("""
============================================
Movie reviews in training set   : {}
Movie reviews in test set       : {}
Movie reviews in validation set : {}
--------------------------------------------
""".format(len(train), len(test), len(valid)))


Movie reviews in training set   : 17500
Movie reviews in test set       : 25000
Movie reviews in validation set : 7500
--------------------------------------------



## Building a vocabulary (dictionary)

In [21]:
voc_len = 25000

tokens.build_vocab(train, max_size=voc_len)
labels.build_vocab(train)

## Building iterators

In [22]:
batch_size = 64

device = torch.device('cpu')
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_itr, valid_itr, test_itr = bi.splits((train, valid, test), 
                                           batch_size = batch_size,
                                           device = device)

# Modeling

## Setup for the model

### Recurrent Neural Networks

The architecture for this network is shown below.

<img src="https://github.com/agungsantoso/deep-learning-v2-pytorch/blob/master/sentiment-rnn/assets/network_diagram.png?raw=1" width=30%>

Source is $\texttt{'/github/bentrevett/pytorch-sentiment-analysis'}$

In [24]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

### Params for the model

In [25]:
input_dim = len(tokens.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1

### Instatiating the model

In [26]:
model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)

## Train the Model

### Fix the optimizer and criterion

In [27]:
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [28]:
criterion = nn.BCEWithLogitsLoss()

### Pass the model to estimate to target device

In [29]:
model = model.to(device)
criterion = criterion.to(device)

### Define a function to store the accuracy of the model  

In [30]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

### ML-alike pipeline 

In [31]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Custom function to mnonitor the execution of the pipeline

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
epoch_count = 5

best_valid_loss = float('inf')

for epoch in range(epoch_count):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_itr, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_itr, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# How to improve the model?

+ ML/DL levers:
  - using different optimizers
  - regularization
+ NLP levers:
  - using pre-trained embeddings
  - using context-aware models of the language (e.g.. BERT and ELMo)